In [1]:
import sqlite3
import utils
import pandas as pd
from sklearn.model_selection import train_test_split
from numba import cuda

from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import numpy as np
import torch

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.layers import Conv1D, GlobalMaxPooling1D,LSTM

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
import keras.backend as K

#dtype='float16'
#K.set_floatx(dtype)

# default is 1e-7 which is too small for float16.  Without adjusting the epsilon, we will get NaN predictions because of divide by zero problems
#K.set_epsilon(1e-3) 

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15921784983877419244
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4557766656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10902205990317183914
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [5]:
def free_gpu_mem():
    cuda.select_device(0)
    cuda.close()

In [6]:
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

In [7]:
word_vec_size=100

In [8]:
def w2v_per_token_wiki(word_vec_size):
    conn=sqlite3.connect('songs.db')
    cur=conn.cursor()
    example_text=cur.execute("select Lyrics from song where Lyrics is not null").fetchall()
    labels=[i[0] for i in cur.execute("select Economic from song where Lyrics is not null").fetchall()]
    songs=[]
    for i in example_text:
        songs.append(utils.tokenize_word(utils.clean_text(i[0])))

    embeddings_index,songs=utils.get_embedding_dict(songs,'glove-twitter-100')
    max_words=len(embeddings_index.keys())
    X,y=np.array(songs),np.array(labels)

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)

    length = []
    for x in X:
        length.append(len(x))

    X = pad_sequences(sequences, maxlen=max(length)+2)

    embedding_matrix = np.zeros((max_words, word_vec_size))
    for word, i in tokenizer.word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.5)
    
    
    return x_train,x_test,y_train,y_test,embedding_matrix,max(length)+2,max_words

In [9]:
x_train,x_test,y_train,y_test,embedding_matrix,max_sent_len,max_words=w2v_per_token_wiki(word_vec_size)

In [10]:
def dummy(x_train,x_test,y_train,y_test):
    dummy = DummyClassifier()
    dummy.fit(x_train, y_train)
    y_pred=dummy.predict(x_test)
    return f1_score(y_test,y_pred)

In [11]:
print(dummy(x_train,x_test,y_train,y_test))

0.42380460956312355


In [12]:
def dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(256, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=256, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [13]:
print(dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 4s - loss: 0.9461 - acc: 0.5777 - val_loss: 0.6032 - val_acc: 0.6424
Epoch 2/100
 - 1s - loss: 0.4831 - acc: 0.7604 - val_loss: 0.5797 - val_acc: 0.6496
Epoch 3/100
 - 1s - loss: 0.3464 - acc: 0.8665 - val_loss: 0.5741 - val_acc: 0.6529
Epoch 4/100
 - 1s - loss: 0.2317 - acc: 0.9384 - val_loss: 0.6039 - val_acc: 0.6511
Epoch 5/100
 - 1s - loss: 0.1492 - acc: 0.9702 - val_loss: 0.6450 - val_acc: 0.6496
Epoch 6/100
 - 1s - loss: 0.1005 - acc: 0.9805 - val_loss: 0.6917 - val_acc: 0.6493
Epoch 7/100
 - 1s - loss: 0.0669 - acc: 0.9871 - val_loss: 0.7386 - val_acc: 0.6523
Epoch 8/100
 - 1s - loss: 0.0539 - acc: 0.9883 - val_loss: 0.7845 - val_acc: 0.6511
Epoch 9/100
 - 1s - loss: 0.0452 - acc: 0.989

Epoch 86/100
 - 1s - loss: 0.0219 - acc: 0.9925 - val_loss: 1.3690 - val_acc: 0.6520
Epoch 87/100
 - 1s - loss: 0.0234 - acc: 0.9934 - val_loss: 1.3755 - val_acc: 0.6532
Epoch 88/100
 - 1s - loss: 0.0222 - acc: 0.9922 - val_loss: 1.3471 - val_acc: 0.6529
Epoch 89/100
 - 1s - loss: 0.0282 - acc: 0.9928 - val_loss: 1.3565 - val_acc: 0.6499
Epoch 90/100
 - 1s - loss: 0.0215 - acc: 0.9928 - val_loss: 1.3580 - val_acc: 0.6541
Epoch 91/100
 - 1s - loss: 0.0266 - acc: 0.9931 - val_loss: 1.3764 - val_acc: 0.6532
Epoch 92/100
 - 1s - loss: 0.0261 - acc: 0.9928 - val_loss: 1.3720 - val_acc: 0.6553
Epoch 93/100
 - 1s - loss: 0.0269 - acc: 0.9931 - val_loss: 1.3797 - val_acc: 0.6556
Epoch 94/100
 - 1s - loss: 0.0220 - acc: 0.9925 - val_loss: 1.3851 - val_acc: 0.6502
Epoch 95/100
 - 1s - loss: 0.0264 - acc: 0.9925 - val_loss: 1.3956 - val_acc: 0.6502
Epoch 96/100
 - 1s - loss: 0.0230 - acc: 0.9931 - val_loss: 1.3980 - val_acc: 0.6499
Epoch 97/100
 - 1s - loss: 0.0212 - acc: 0.9937 - val_loss: 1.403

In [14]:
def dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(256, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=256, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [15]:
print(dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 4s - loss: 1.3431 - acc: 0.5326 - val_loss: 0.6515 - val_acc: 0.6016
Epoch 2/100
 - 1s - loss: 0.6004 - acc: 0.6372 - val_loss: 0.6147 - val_acc: 0.6436
Epoch 3/100
 - 1s - loss: 0.5009 - acc: 0.7583 - val_loss: 0.5964 - val_acc: 0.6451
Epoch 4/100
 - 1s - loss: 0.3871 - acc: 0.8437 - val_loss: 0.5763 - val_acc: 0.6418
Epoch 5/100
 - 1s - loss: 0.2769 - acc: 0.9134 - val_loss: 0.5966 - val_acc: 0.6544
Epoch 6/100
 - 1s - loss: 0.1939 - acc: 0.9468 - val_loss: 0.6383 - val_acc: 0.6572
Epoch 7/100
 - 1s - loss: 0.1289 - acc: 0.9720 - val_loss: 0.7145 - val_acc: 0.6496
Epoch 8/100
 - 1s - loss: 0.0874 - acc: 0.9808 - val_loss: 0.7403 - val_acc: 0.6478
Epoch 9/100
 - 1s - loss: 0.0680 - acc: 0.9865 - val_loss: 0.7849 - val_acc: 0.6487
Epoch 10/100
 - 1s - loss: 0.0499 - acc: 0.9901 - val_loss: 0.8250 - val_acc: 0.6556
Epoch 11/100
 - 1s - loss: 0.0457 - acc: 0.9904 - val_loss: 0.8669 - val_acc: 0.6553
Epoch 12/100
 - 1s - loss:

Epoch 97/100
 - 1s - loss: 0.0173 - acc: 0.9922 - val_loss: 1.5635 - val_acc: 0.6562
Epoch 98/100
 - 1s - loss: 0.0180 - acc: 0.9919 - val_loss: 1.5796 - val_acc: 0.6544
Epoch 99/100
 - 1s - loss: 0.0130 - acc: 0.9919 - val_loss: 1.6074 - val_acc: 0.6538
Epoch 100/100
 - 1s - loss: 0.0157 - acc: 0.9928 - val_loss: 1.6138 - val_acc: 0.6553
0.6105263157894737


In [16]:
def cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=256, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [17]:
print(cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 4s - loss: 0.7560 - acc: 0.5089 - val_loss: 0.6879 - val_acc: 0.5694
Epoch 2/100
 - 1s - loss: 0.6951 - acc: 0.5618 - val_loss: 0.6715 - val_acc: 0.5790
Epoch 3/100
 - 1s - loss: 0.6677 - acc: 0.5960 - val_loss: 0.6661 - val_acc: 0.6106
Epoch 4/100
 - 1s - loss: 0.6516 - acc: 0.6189 - val_loss: 0.6663 - val_acc: 0.6004
Epoch 5/100
 - 1s - loss: 0.6234 - acc: 0.6592 - val_loss: 0.6442 - val_acc: 0.6319
Epoch 6/100
 - 1s - loss: 0.5923 - acc: 0.7063 - val_loss: 0.6288 - val_acc: 0.6280
Epoch 7/100
 - 1s - loss: 0.5634 - acc: 0.7217 - val_loss: 0.6332 - val_acc: 0.6190
Epoch 8/100
 - 1s - loss: 0.5101 - acc: 0.7839 - val_loss: 0.6118 - val_acc: 0.6463
Epoch 9/100
 - 1s - loss: 0.4637 - acc: 0.8127 - val_loss: 0.5984 - val_acc: 0.6376
Epoch 10/100
 - 1s - loss: 0.4139 - acc: 0.8452 - val_loss: 0.6020 - val_acc: 0.6463
Epoch 11/100
 - 1s - loss: 0.3662

Epoch 94/100
 - 1s - loss: 0.0218 - acc: 0.9916 - val_loss: 1.2235 - val_acc: 0.6572
Epoch 95/100
 - 1s - loss: 0.0251 - acc: 0.9910 - val_loss: 1.2188 - val_acc: 0.6538
Epoch 96/100
 - 1s - loss: 0.0281 - acc: 0.9904 - val_loss: 1.2554 - val_acc: 0.6529
Epoch 97/100
 - 1s - loss: 0.0262 - acc: 0.9916 - val_loss: 1.2336 - val_acc: 0.6550
Epoch 98/100
 - 1s - loss: 0.0213 - acc: 0.9910 - val_loss: 1.3166 - val_acc: 0.6541
Epoch 99/100
 - 1s - loss: 0.0235 - acc: 0.9922 - val_loss: 1.2491 - val_acc: 0.6517
Epoch 100/100
 - 1s - loss: 0.0236 - acc: 0.9925 - val_loss: 1.2823 - val_acc: 0.6466
0.6343283582089553


In [18]:
def cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=256, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [19]:
print(cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 2s - loss: 0.7205 - acc: 0.5305 - val_loss: 0.6838 - val_acc: 0.5664
Epoch 2/100
 - 1s - loss: 0.6759 - acc: 0.5774 - val_loss: 0.6603 - val_acc: 0.5769
Epoch 3/100
 - 1s - loss: 0.6203 - acc: 0.6682 - val_loss: 0.6395 - val_acc: 0.6352
Epoch 4/100
 - 1s - loss: 0.5720 - acc: 0.7190 - val_loss: 0.6229 - val_acc: 0.6331
Epoch 5/100
 - 1s - loss: 0.5048 - acc: 0.7905 - val_loss: 0.5941 - val_acc: 0.6484
Epoch 6/100
 - 1s - loss: 0.4201 - acc: 0.8611 - val_loss: 0.5808 - val_acc: 0.6487
Epoch 7/100
 - 1s - loss: 0.3292 - acc: 0.9059 - val_loss: 0.5916 - val_acc: 0.6547
Epoch 8/100
 - 1s - loss: 0.2432 - acc: 0.9447 - val_loss: 0.6005 - val_acc: 0.6484
Epoch 9/100
 - 1s - loss: 0.1701 - acc: 0.9696 - val_loss: 0.6202 - val_acc: 0.6508
Epoch 10/100
 - 1s - loss: 0.1193 - acc: 0.9835 - val_loss: 0.6534 - val_acc: 0.6496
Epoch 11/100
 - 1s - loss: 0.0841 - acc: 0.9880 - val_loss: 0.6854 - val_acc: 0.6532
Epoch 12/100
 - 1s - loss:

Epoch 97/100
 - 1s - loss: 0.0163 - acc: 0.9922 - val_loss: 1.0373 - val_acc: 0.6541
Epoch 98/100
 - 1s - loss: 0.0172 - acc: 0.9925 - val_loss: 1.0504 - val_acc: 0.6379
Epoch 99/100
 - 1s - loss: 0.0191 - acc: 0.9910 - val_loss: 1.0111 - val_acc: 0.6490
Epoch 100/100
 - 1s - loss: 0.0151 - acc: 0.9922 - val_loss: 1.0249 - val_acc: 0.6463
0.5773788150807899


In [20]:
def lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=256, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [21]:
print(lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 118s - loss: 0.6964 - acc: 0.5299 - val_loss: 0.6850 - val_acc: 0.5667
Epoch 2/10
 - 115s - loss: 0.6835 - acc: 0.5609 - val_loss: 0.6804 - val_acc: 0.5676
Epoch 3/10
 - 116s - loss: 0.6771 - acc: 0.5660 - val_loss: 0.6763 - val_acc: 0.5760
Epoch 4/10
 - 116s - loss: 0.6684 - acc: 0.5930 - val_loss: 0.6712 - val_acc: 0.5814
Epoch 5/10
 - 116s - loss: 0.6596 - acc: 0.6102 - val_loss: 0.6642 - val_acc: 0.5889
Epoch 6/10
 - 117s - loss: 0.6421 - acc: 0.6297 - val_loss: 0.6640 - val_acc: 0.5928
Epoch 7/10
 - 118s - loss: 0.6352 - acc: 0.6318 - val_loss: 0.6652 - val_acc: 0.5886
Epoch 8/10
 - 118s - loss: 0.6180 - acc: 0.6604 - val_loss: 0.6567 - val_acc: 0.6097
Epoch 9/10
 - 118s - loss: 0.5979 - acc: 0.6925 - val_loss: 0.6508 - val_acc: 0.6061
Epoch 10/10
 - 119s - loss: 0.5650 - acc: 0.7075 - val_loss: 0.6478 - val_acc: 0.6217
0.5137118578601777


In [22]:
def lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=256, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [23]:
print(lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 135s - loss: 0.6893 - acc: 0.5509 - val_loss: 0.6838 - val_acc: 0.5628
Epoch 2/10
 - 132s - loss: 0.6784 - acc: 0.5723 - val_loss: 0.6774 - val_acc: 0.5751
Epoch 3/10
 - 133s - loss: 0.6633 - acc: 0.5999 - val_loss: 0.6712 - val_acc: 0.5841
Epoch 4/10
 - 132s - loss: 0.6400 - acc: 0.6381 - val_loss: 0.6611 - val_acc: 0.5847
Epoch 5/10
 - 133s - loss: 0.6164 - acc: 0.6739 - val_loss: 0.6603 - val_acc: 0.5919
Epoch 6/10
 - 133s - loss: 0.5818 - acc: 0.7009 - val_loss: 0.6560 - val_acc: 0.6175
Epoch 7/10
 - 132s - loss: 0.5153 - acc: 0.7469 - val_loss: 0.6917 - val_acc: 0.6340
Epoch 8/10
 - 132s - loss: 0.4383 - acc: 0.7926 - val_loss: 0.7348 - val_acc: 0.6400
Epoch 9/10
 - 132s - loss: 0.3806 - acc: 0.8290 - val_loss: 0.7237 - val_acc: 0.6346
Epoch 10/10
 - 132s - loss: 0.3146 - acc: 0.8644 - val_loss: 0.8135 - val_acc: 0.6409
0.5826056584002794
